In [1]:
# 英文编程书籍笔记处理——《Getting Started with Processing.py》
import pathlib
import re


path_txt = pathlib.Path(r"D:\Python_Work\reading_notes_cleaner\learn python visually.txt")
path_md = pathlib.Path(r"D:\Python_Work\reading_notes_cleaner\learn python visually.md")

In [2]:
contents = []
code_block = False

with open(path_txt, encoding='utf-8') as f:
    for line in f:
        if re.match(r'\d\d\d\d-\d\d-\d\d.*', line):   # 去除笔记时间标记
            line = ''
        # 代码块未结束
        code_closed = False
        if code_block:  
            code_closed = False
            if re.search(r'  +', line.strip()): # 行内容出现2个以上英文空格, 则增加换行
                pre_space = re.match(' *', line).group()
                line = pre_space + re.sub(r'  +[^#]', lambda x: '\n' + x.group(), line.strip()) + '\n'
            line = re.sub(r' def', '\ndef', line)
            if re.match(r'\d{1,2}/[A-Z]', line) or re.match(r'[A-Z]', line):    # 大写字母开头或章节开始视为代码块结束
                code_closed = True
                code_block = False
        # 非代码块
        if code_closed or not code_block:
            # 增加代码块标记
            if re.match(r'[a-z#]', line):    # 小写字母开头视为代码块开始
                code_block = True
                line = "```python\n" + line
            elif re.match(r'\d{1,2}/[A-Z]', line) or line.startswith('Chapter '):  # 增加章节标记
                line = '## ' + line   
            elif re.match(r'[A-Z]', line) and not re.search(r'[.!?:]$', line) and len(line.split(' ')) < 10:    # 大写字母开头 且 不以标点结尾 且 单词数小于10个，加粗处理
                # line = '**' + line.strip() + '**\n'
                if line.startswith('Example'):
                    line = '#### ' + line
                else:
                    line = '### ' + line
            if code_closed:
                line = "```\n" + line

        contents.append(line)
        
content = ''.join(contents)


In [3]:
with open(path_md, 'w', encoding='utf-8') as f:
    f.write(content)